In [1]:
import twint
import pandas as pd
import numpy as np
import nest_asyncio
nest_asyncio.apply()

In [2]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
get_ipython().run_line_magic("matplotlib", "inline")

In [3]:
# Create an empty dataframe
df = pd.DataFrame()
# Initialize empty source list
source = []
# Define previous dataframe length (i.e. number of rows)
previous_df_len = 0

my_list = ["inflation_guy", "cnbc", "dylanmatt", "bloomberg", "wsj"]
for tweets in my_list:
    c = twint.Config()
    c.Search = "inflation"
    c.Hide_output = True
    c.Lang= "en"
    c.Pandas = True
    c.Since = "2020-01-01"
    c.near = "united_states"
    c.Limit = 5000
    twint.run.Search(c)
    df = pd.concat([df,twint.output.panda.Tweets_df], axis=0)
    source.append(list(np.repeat(tweets,len(df)-previous_df_len)))
    previous_df_len = len(df)

In [4]:
# flatten source list (from a list of list into a single list)
flat_source = [item for sublist in source for item in sublist]

In [5]:
Tweets_df = df
Tweets_df['source_tweet'] = flat_source
Tweets_df.head()

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,source_tweet
0,1443375294220488705,1443375294220488705,1.632962e+12,2021-09-29 20:41:01,-0400,,Shocking spike in September inflation; may fee...,en,[],[rsx],...,,,,,[],,,,,inflation_guy
1,1443375291687112705,1443337938667020293,1.632962e+12,2021-09-29 20:41:00,-0400,,"@ChuckGrassley With ""Inflation"" is a lot more ...",en,[],[],...,,,,,"[{'screen_name': 'ChuckGrassley', 'name': 'Chu...",,,,,inflation_guy
2,1443375241166835712,1443286306142965763,1.632962e+12,2021-09-29 20:40:48,-0400,,@SouadH9 @X345__ @cjeanfr @BotInflation Oui jp...,fr,[],[],...,,,,,"[{'screen_name': 'SouadH9', 'name': 'math dest...",,,,,inflation_guy
3,1443375220400824322,1443366922783952896,1.632962e+12,2021-09-29 20:40:43,-0400,,@Craigor47238257 @gatewaypundit Good luck Corr...,en,[],[],...,,,,,"[{'screen_name': 'Craigor47238257', 'name': '🇺...",,,,,inflation_guy
4,1443375192336572417,1443368728335290368,1.632962e+12,2021-09-29 20:40:37,-0400,,I would like to see Australian Labor win the n...,en,[auspol],[],...,,,,,[],,,,,inflation_guy


In [6]:
Tweets_df['hashtags']

0             []
1             []
2             []
3             []
4       [auspol]
          ...   
4999          []
5000          []
5001          []
5002          []
5003          []
Name: hashtags, Length: 25041, dtype: object

In [7]:
Tweets_df.drop(['id', 'conversation_id', 'created_at', 'timezone', 'place', 'language','cashtags', 'user_id', 'user_id_str','username', 'day', 'hour', 'link', 'urls', 'photos', 'video', 'thumbnail', 'retweet', 'quote_url', 'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'], axis = 1, inplace = True)

In [8]:
Tweets_df.head()

,date,tweet,hashtags,name,nlikes,nreplies,nretweets,source_tweet
0,2021-09-29 20:41:01,Shocking spike in September inflation; may fee...,[],TalkMarkets,0,0,0,inflation_guy
1,2021-09-29 20:41:00,"@ChuckGrassley With ""Inflation"" is a lot more ...",[],Matthew (马修)/(マシュー),0,0,0,inflation_guy
2,2021-09-29 20:40:48,@SouadH9 @X345__ @cjeanfr @BotInflation Oui jp...,[],NavNico 🌿,0,0,0,inflation_guy
3,2021-09-29 20:40:43,@Craigor47238257 @gatewaypundit Good luck Corr...,[],Fred Johnson,0,0,0,inflation_guy
4,2021-09-29 20:40:37,I would like to see Australian Labor win the n...,[auspol],FullyvaccinatedGuy,1,0,0,inflation_guy


In [9]:
#Download Vader/Update the VADER Lexicon
nltk.download("vader_lexicon")

#Initiate the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\afuen\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [10]:
# Sentiment calculation based on compound score
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    return result

In [ ]:
# Sentiment scores dictionaries
tweet_sent = {
    "tweet_compound": [],
    "tweet_pos": [],
    "tweet_neu": [],
    "tweet_neg": [],
    "tweet_sent": [],

}

# Get sentiment for the text and the title
for index, row in Tweets_df.iterrows():
    try:
        # Sentiment scoring with VADER
        tweet_sentiment = analyzer.polarity_scores(row["tweet"])
        tweet_sent["tweet_compound"].append(tweet_sentiment["compound"])
        tweet_sent["tweet_pos"].append(tweet_sentiment["pos"])
        tweet_sent["tweet_neu"].append(tweet_sentiment["neu"])
        tweet_sent["tweet_neg"].append(tweet_sentiment["neg"])
        tweet_sent["tweet_sent"].append(get_sentiment(tweet_sentiment["compound"]))

    
    except AttributeError:
        pass

# Attaching sentiment columns to the News DataFrame
tweet_sentiment_df = pd.DataFrame(tweet_sent)
tweet_sentiment_df.head()

In [ ]:
def join_dataframes(df1,df2): 
    df1=df1.join(df2)
    return df1

In [ ]:
Tweets_dataframe=join_dataframes(Tweets_df, tweet_sentiment_df)
Tweets_dataframe.columns

In [ ]:
Tweets_dataframe.drop(['hashtags', 'name', 'nlikes', 'nreplies', 'nretweets'], axis = 1, inplace = True)

In [ ]:
Tweets_dataframe.head()

In [ ]:
Tweets_dataframe['date'] = pd.to_datetime(Tweets_dataframe['date'], format ='%Y-%m-%d %H:%M:%S')
Tweets_dataframe.head()

In [ ]:
import hvplot.pandas
Tweets_dataframe.sort_values('date').hvplot.line(
"date",
"tweet_compound",
xlabel="Date",
ylabel="Sentiment Score",
groupby="source_tweet",
title="Tweets Score by Date")

In [ ]:
Tweets_dataframe.groupby('source_tweet').mean()

In [ ]:
Tweets_dataframe.describe()